# Creating a BSP for the `ipython_microblaze` Library

Board support packages or BSPs are foundation of the `ipython_microblaze` ecosystem. BSPs live in the `$INSTALL/bsp` directory and two are provided by default for the PYNQ PMOD and Arduino systems. A BSP combines a set of header files, the core runtime libraries and a linker script to assemble the image.

Xilinx SDK is used to create BSPs however some changes to the default settings are needed to make things work with ipython_microblaze. To simplify the process a script is created that will create bsps in a suitable form. First the HDF file needs to be exported from Vivado using the _Export -> Export Hardware_ menu command. The HDF file can then be passed to the `bsp_gen/generate_bsps.sh` script included in this repository. This script has been tested with SDK 2016.1 and there are no guarantees with any other version.

Assuming the script works, the output should a `bsp` directory in the current working directory inside of which is a directory for each microblaze in the design. These directories are the ipython_microblaze BSPs and can be copied into the bsp location mentioned above.

In PYNQ microblaze subsystems are described by an `mb_info` dictionary. If we load the base overlay bitstream but without the  `BaseOverlay` class we can see how this works for any design.

In [1]:
import pynq

base = pynq.Overlay('base.bit')

Microblaze subsystems are determined by hierarchy. In this case the `iop1`, `iop2` and `iop3` hierarchies are instances of `MicroblazeHierarchy` which will expose the `mb_info`

In [2]:
base.iop1.mb_info

{'intr_ack_name': 'mb_iop1_intr_ack',
 'intr_pin_name': 'iop1/dff_en_reset_0/q',
 'ip_name': 'iop1/mb_bram_ctrl',
 'mbtype': 'Unknown',
 'rst_name': 'mb_iop1_reset'}

The ipython_microblaze library uses the `mbtype` entry in this dict to determine which BSP to load so we can select our BSP by changing it. In this case we are going to use the built in `Pmod` BSP.

In [3]:
base.iop1.mb_info['mbtype'] = 'Pmod'

Now we can import the `ipython_microblaze` library and use the microblaze from Jupyter

In [4]:
import ipython_microblaze

In [5]:
%%microblaze base.iop1 hello_world
#include <xil_io.h>

int main() {
    print("Hello, World!\n");
}

In [6]:
hello_world.stream.read()

b'Hello, World!\n'